In [9]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import os, sys
import random, math, time
import shutil
import numpy as np
import matplotlib.pyplot as plt

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print("device:", torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        device = torch.device("cpu")
        print("device: cpu")
    return device
device = get_device()

LEARNING_RATE = 0.001
MOMENTUM = 0.9
EPOCHS = 10
BATCH_SIZE = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def convert_to_imshow_format(image):
    # first convert back to [0,1] range from [-1,1] range
    image = image / 2 + 0.5
    image = image.numpy()
    # convert from CHW to HWC
    # from 3x32x32 to 32x32x3
    return image.transpose(1,2,0)

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    weights = torch.zeros(4, 10, 64)  # [b, 10, hid_dim]
    for idx, (inputs, targets) in enumerate(iterator):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs, features = model(inputs, weights)
        weights = model.state_dict()['attention.fc_k.weight']

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = outputs.max(1)                  # probs, index = torch.max(outputs.data, dim=1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()  # correct += (predicted == labels).sum().item()

    return epoch_loss/len(iterator), 100.*correct/total

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct = 0
    total = 0
    weights = torch.zeros(4, 10, 64) # [b, 10, hid_dim]
    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(iterator):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs, features = model(inputs, weights)
            weights = model.state_dict()['attention.fc_k.weight']

            loss = criterion(outputs, targets)

            epoch_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return epoch_loss/len(iterator), 100.*correct/total

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


# In[ ]: prepare
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.ImageFolder('/home/linux/data/Imagenet/train', transform=transform)
testset = torchvision.datasets.ImageFolder('/home/linux/data/Imagenet/val', transform=transform)

loaders = {
    'train' : torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True),
    'test'  : torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False),
}

total_train_batch = len(loaders['train'])
total_test_batch = len(loaders['test'])
print('train 총 배치의 수 : {}'.format(total_train_batch))
print('test 총 배치의 수 : {}'.format(total_test_batch))
########################################################################################################################
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            #3 224 224
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #64 112 112
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #128 56 56
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #256 28 28
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            #512 14 14
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
            #512 7 7
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
########################################################################################################################
# 모델 정의하기
model = VGG().to(device)
print(model)

# 손실 함수와 Optimizer 정의하기
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# In[ ]: train
checkpoint_path = './checkpoint/{}_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
best_model_path = './checkpoint/{}_best_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
best_valid_loss = float('inf')

## IF you want to transfer learning 
# model, optimizer, epoch, valid_loss_min = load_ckp(checkpoint_path, model, optimizer)
# print("model = ", model)
# print("optimizer = ", optimizer)
# print("epoch = ", epoch)
# print("valid_loss_min = {:.4f}".format(valid_loss_min))

for epoch in range(EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, loaders['train'], optimizer, criterion)
    valid_loss, val_acc = evaluate(model, loaders['test'], criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    val_loss_history.append(valid_loss)
    val_acc_history.append(val_acc)

    print('Epoch: {} | Time: {}m {}s'.format(epoch + 1, epoch_mins, epoch_secs))
    print('\tTrain Loss: {:.3f} | Acc: {:.2f}%'.format(train_loss, train_acc))
    print('\t Val. Loss: {:.3f} | Acc: {:.2f}%'.format(valid_loss, val_acc))

    # create checkpoint variable and add important data
    checkpoint = {
        'epoch': epoch + 1,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    # save checkpoint
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    if valid_loss < best_valid_loss:
        print('Val. loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(best_valid_loss, valid_loss))
        best_valid_loss = valid_loss
        # torch.save(model.state_dict(), BEST_PATH)
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)

print('Finished Training')
# torch.save(model.state_dict(), SAVE_PATH)

# In[ ]: test
checkpoint_path = './checkpoint/{}_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
best_model_path = './checkpoint/{}_best_model_{}_epoch_{}.pth'.format('ImageNet', 'VGG', EPOCHS)
# model.load_state_dict(torch.load(SAVE_PATH))

model, optimizer, epoch, valid_loss_min = load_ckp(checkpoint_path, model, optimizer)
print("model = ", model)
print("optimizer = ", optimizer)
print("epoch = ", epoch)
print("valid_loss_min = {:.4f}".format(valid_loss_min))

test_loss, test_acc = evaluate(model, loaders['test'], criterion)
print('| Test Loss: {:.3f} | Acc: {:.2f}%'.format(test_loss, test_acc))

x_len = np.arange(len(train_loss_history))
plt.plot(x_len, train_loss_history, marker='.', c='blue', label="Train-set Loss")
plt.plot(x_len, val_loss_history, marker='.', c='red', label="Val-set Loss")
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# In[ ]: test II
dataiter = iter(loaders['test'])
images, labels = dataiter.next()
fig, axes = plt.subplots(1, len(images), figsize=(12, 2.5))
for idx, image in enumerate(images):
    axes[idx].imshow(convert_to_imshow_format(image))
    axes[idx].set_title(classes[labels[idx]])
    axes[idx].set_xticks([])
    axes[idx].set_yticks([])
plt.show()

outputs, features = model(images.to(device))
# outputs, features, attention = model(images.to(device))
sm = nn.Softmax(dim=1)
sm_outputs = sm(outputs)
print("outputs: ", outputs, outputs.size())
print("prediction: ", sm_outputs, sm_outputs.size())
print("features: ", features, features.size())

probs, index = torch.max(sm_outputs, dim=1)
for p, i in zip(probs, index):
    print('{} - {:.4f}'.format(classes[i], p))

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for idx, (images, labels) in enumerate(loaders['test']):
        images, labels = images.to(device), labels.to(device)
        outputs, features, _ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1

model_accuracy = total_correct / total_images * 100
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))

print('{0:10s} - {1}'.format('Category','Accuracy'))
for i, r in enumerate(confusion_matrix):
    print('{0:10s} - {1:.1f}'.format(classes[i], r[i]/np.sum(r)*100))

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.matshow(confusion_matrix, aspect='auto', vmin=0, vmax=1000, cmap=plt.get_cmap('Blues'))
plt.ylabel('Actual Category')
plt.yticks(range(10), classes)
plt.xlabel('Predicted Category')
plt.xticks(range(10), classes)
plt.show()

device: cpu
train 총 배치의 수 : 160146
test 총 배치의 수 : 6250
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0,

TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
batch 256 -> 128 -> 64
